In [6]:
from pathlib import Path

In [7]:
test_folder = Path('./test')

In [8]:
[x for x in test_folder.iterdir() if test_folder.is_dir()]

[WindowsPath('test/frieren_clip1.mp4')]

In [9]:
path = Path('./test')

# path.iterdir()
[x for x in test_folder.iterdir() if test_folder.is_dir()]

[WindowsPath('test/frieren_clip1.mp4')]

In [10]:
import shutil
dst = Path("../frontend/src/assets")

for file in test_folder.iterdir():
    if file.is_file():
        shutil.move(file, dst / file.name)

In [11]:
def move_files(src: Path, dst: Path):
    for file in src.iterdir():
        if file.is_file():
            shutil.move(file, dst / file.name)

In [12]:
move_files(Path("./test"), Path("../frontend/src/assets"))

## Testing out scenedetect

In [66]:
from scenedetect import *
print(f"Finding path..")
path = "./test/frieren_clip1.mp4"

print(f"Finding cuts...")
scene_list = detect(path, 
                    AdaptiveDetector(
                        adaptive_threshold=4.0,
                        min_scene_len=25, 
                        min_content_val=35.0 
                    )
                )

print(f"SCENE LIST:\n {scene_list}")

print(f"Splitting scenes...")
split_video_ffmpeg(path, scene_list, output_dir="./output_test")

Finding path..
Finding cuts...
SCENE LIST:
 [(00:00:00.000 [frame=0, fps=23.976], 00:00:02.502 [frame=60, fps=23.976]), (00:00:02.502 [frame=60, fps=23.976], 00:00:05.505 [frame=132, fps=23.976]), (00:00:05.505 [frame=132, fps=23.976], 00:00:10.510 [frame=252, fps=23.976]), (00:00:10.510 [frame=252, fps=23.976], 00:00:13.513 [frame=324, fps=23.976]), (00:00:13.513 [frame=324, fps=23.976], 00:00:23.941 [frame=574, fps=23.976])]
Splitting scenes...


0

In [67]:
from scenedetect import SceneManager, open_video, split_video_ffmpeg
from scenedetect.detectors import ContentDetector, AdaptiveDetector
from scenedetect.scene_manager import Interpolation

path = "./test/frieren_clip1.mp4"

video = open_video(path)

scene_manager = SceneManager()    
scene_manager.interpolation = Interpolation.AREA    

scene_manager.clear_detectors()

"""ContentDetector Settings"""
# scene_manager.add_detector(
#     ContentDetector(
#         threshold=32.0,
#         min_scene_len=30
#     )
# )

"""AdaptiveDetector Settings"""
scene_manager.add_detector(
    AdaptiveDetector(
        adaptive_threshold=3.5,
        min_scene_len=10,
        min_content_val=30.0
    )
)
scene_manager.downscale = 3


scene_manager.detect_scenes(frame_source=video)
scene_list = scene_manager.get_scene_list()

print(f"Splitting scenes..")
print(scene_list)
split_video_ffmpeg(path, scene_list, output_dir="./output_test")

Downscale factor will be ignored because auto_downscale=True!


Splitting scenes..
[(00:00:00.000 [frame=0, fps=23.976], 00:00:00.500 [frame=12, fps=23.976]), (00:00:00.500 [frame=12, fps=23.976], 00:00:02.502 [frame=60, fps=23.976]), (00:00:02.502 [frame=60, fps=23.976], 00:00:05.505 [frame=132, fps=23.976]), (00:00:05.505 [frame=132, fps=23.976], 00:00:10.510 [frame=252, fps=23.976]), (00:00:10.510 [frame=252, fps=23.976], 00:00:13.513 [frame=324, fps=23.976]), (00:00:13.513 [frame=324, fps=23.976], 00:00:17.017 [frame=408, fps=23.976]), (00:00:17.017 [frame=408, fps=23.976], 00:00:23.941 [frame=574, fps=23.976])]


0

### Keyframe System Design

1) Generate keyframes from the video using FFMPEG and subprocess
2) Use those keyframes to trim the video around each keyframe, 1 second around
3) Use scenedetect on the updated video

In [5]:
import subprocess
import json

def preprocess_video(input_path, output_path):
    cmd = [
        "ffmpeg", "-y",
        "-i", input_path,
        "-vf", "scale=640:-1,format=gray,boxblur=2:1",
        output_path
    ]
    subprocess.run(
        cmd,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        check=True
    )
def generate_keyframes(video_path: str):
    """Generates keyframe for a given video"""
    cmd = [
        "ffprobe",                      # Launches the ffprobe executable
        "-skip_frame", "nokey",         # ONLY look at keyframes, skip others (speed)
        "-select_streams", "v:0",       # Only looks at the first video stream
        "-show_frames",                 # Shows metadata for EVERY frame
        "-show_entries", "frame=best_effort_timestamp_time",  # Filter, we only get the frame of the data
        "-of", "json",                  # Export as json format
        video_path
    ]

    print(f"Execting command..")
    # Executing system command
    result = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,  # Return strings instead of bytes
        check=True  # Crash if the command fails
    )

    data = json.loads(result.stdout)
    
    return [
        float(frame["best_effort_timestamp_time"])
        for frame in data.get("frames", [])
        if "best_effort_timestamp_time" in frame
    ]

In [6]:
def keyframe_windows(keyframes, radius=1.0):
    """Generates keyframe windows for """
    windows = [(max(0, key - radius), key + radius) for key in keyframes]

    windows.sort()
    merged = [windows[0]]  
    
    # Ensures that none of the windows overlap each other
    for start, end in windows[1:]:
        last_start, last_end = merged[-1]
        if start <= last_end:
            merged[-1] = (last_start, max(last_end, end)) # Replace appended start with the last end
        else:
            merged.append((start, end))
    
    return merged

In [7]:
import os
import subprocess

def trim_keyframes(video_path: str, output_dir="./keyframe_clips", radius=1.0):
    os.makedirs(output_dir, exist_ok=True)

    # generating keyframes
    print(f"Generating keyframes..")
    keyframes = generate_keyframes(video_path)
    if not keyframes:
        return []
    print(f"Keyframes: {keyframes}")

    # generating keyframe windows
    print(f"Generating keyframe windows...")
    windows = keyframe_windows(keyframes, radius)
    print(f"Windows: {windows}")
    clips = []
    
    print(f"Trimming video from keyframes..")
    # going through each keyframe window to trim the video
    for i, (start, end) in enumerate(windows):
        out_path = os.path.join(output_dir, f"kf_clip_{i:04d}.mp4")

        cmd = [
            "ffmpeg",
            "-y",
            "-ss", str(start),  # trim from start of keyframe
            "-to", str(end),    # trim until end of keyframe 
            "-i", video_path,
            "-c", "copy",
            out_path
        ]

        subprocess.run(
            cmd,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            check=True
        )

        clips.append(
            {
                "clip_path": out_path,
                "window_start": start,
                "window_end": end
            }
        )
    return clips

In [8]:
from scenedetect import SceneManager, open_video
from scenedetect.detectors import AdaptiveDetector, ContentDetector

def detect_and_trim_scenes(
        original_video_path: str,
        keyframe_clips: list,
        threshold,
        output_dir="./output_test",
):
    os.makedirs(output_dir, exist_ok=True)

    final_scenes = []
    scene_idx = 0

    for clip in keyframe_clips:
        clip_path = clip["clip_path"]
        offset = clip["window_start"]
        clip_duration = clip["window_end"] - clip["window_start"]

        video = open_video(clip_path) 
        manager = SceneManager()
        manager.add_detector(
            AdaptiveDetector(
                adaptive_threshold=threshold,
                min_scene_len=10,
                min_content_val=30.0
            )
        )
        manager.detect_scenes(video) # detecting all the keyframes here
        
        scenes = manager.get_scene_list()
        print(f"SCENES: {scenes}")
        
        scene_timestamps = [0.0]
        scene_timestamps += [scene[0].get_seconds() for scene in scenes]
        scene_timestamps.append(clip_duration)

        print(f"SCENE_TIMESTAMPS: {scene_timestamps}")
        for i in range(len(scene_timestamps) - 1):
            abs_start = offset + scene_timestamps[i]
            abs_end = offset + scene_timestamps[i + 1]

            if abs_start >= abs_end:
                print(f"Detected duplicate timestamps: {abs_start}, {abs_end}")
                continue
            out_path = os.path.join(output_dir, f"scene_{scene_idx:04d}.mp4")

            cmd = [
                "ffmpeg", "-y",
                "-ss", str(abs_start),
                "-to", str(abs_end),
                "-i", original_video_path,
                "-c", "copy",
                out_path
            ]

            subprocess.run(cmd,
                           stdout=subprocess.DEVNULL,
                           stderr=subprocess.DEVNULL,
                           check=True)

            final_scenes.append({
                "scene_index": scene_idx,
                "start": abs_start,
                "end": abs_end,
                "path": out_path
            })

            scene_idx += 1
    return final_scenes

In [9]:
import shutil
import time

video_path = "./test/frieren_clip1.mp4"
preprocessed_path = "./test/preprocessed_frieren_clip1.mp4"
preprocess_video(video_path, preprocessed_path)

trimmed_keyframes = trim_keyframes(preprocessed_path, radius=0.3)
print(f"Using trimmed clips for scene detection...")
detect_and_trim_scenes(original_video_path=video_path,
                       keyframe_clips=trimmed_keyframes,
                       threshold=2.0,
                       output_dir="./output_test")
time.sleep(0.2)
shutil.rmtree("./keyframe_clips", ignore_errors=True)

Generating keyframes..
Execting command..
Keyframes: [0.0, 1.501502, 2.502503, 3.503504, 4.462796, 5.505506, 6.631632, 8.008008, 9.300968, 10.510511, 13.513514, 17.017017, 20.520521]
Generating keyframe windows...
Windows: [(0, 0.3), (1.2015019999999998, 1.801502), (2.202503, 2.8025029999999997), (3.203504, 3.8035039999999998), (4.162796, 4.762796), (5.205506, 5.805505999999999), (6.331632, 6.931632), (7.708008, 8.308008000000001), (9.000967999999999, 9.600968), (10.210510999999999, 10.810511), (13.213514, 13.813514000000001), (16.717017, 17.317017), (20.220520999999998, 20.820521)]
Trimming video from keyframes..
Using trimmed clips for scene detection...
SCENES: []
SCENE_TIMESTAMPS: [0.0, 0.3]
SCENES: []
SCENE_TIMESTAMPS: [0.0, 0.6000000000000001]
SCENES: []
SCENE_TIMESTAMPS: [0.0, 0.5999999999999996]
SCENES: []
SCENE_TIMESTAMPS: [0.0, 0.5999999999999996]
SCENES: []
SCENE_TIMESTAMPS: [0.0, 0.5999999999999996]
SCENES: []
SCENE_TIMESTAMPS: [0.0, 0.5999999999999996]
SCENES: []
SCENE_TIM

In [ ]:
import os
import subprocess

def ffmpeg_scene_detection(video_path, threshold=0.3, output_dir="./output_test"):
    os.makedirs(output_dir, exist_ok=True)
    
    # First pass: Detect scenes using FFmpeg's scene detection filter
    detect_cmd = [
        "ffmpeg", "-i", video_path,
        "-vf", f"select='gt(scene,{threshold})',showinfo",
        "-vsync", "vfr",
        "-f", "null", "-"
    ]
    result = subprocess.run(detect_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    # Parse detected scene timestamps from FFmpeg's stderr output
    scenes = []
    for line in result.stderr.splitlines():
        if "pts_time" in line:
            time_str = line.split("pts_time:")[1].split(" ")[0]
            scenes.append(float(time_str))
    
    # Add the start and end of the video to the scene list
    scenes = [0.0] + scenes
    duration_cmd = [
        "ffprobe", "-i", video_path, "-show_entries", "format=duration",
        "-v", "quiet", "-of", "csv=p=0"
    ]
    duration_result = subprocess.run(duration_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    video_duration = float(duration_result.stdout.strip())
    scenes.append(video_duration)
    
    corr = cosine_similarity(edges_t, edges_t_minus_1)
    
    print(f"Detected scenes: {scenes}")
    
    # Second pass: Cut the video into segments based on the detected scenes
    for i in range(len(scenes) - 1):
        start_time = scenes[i]
        end_time = scenes[i + 1]
        out_path = os.path.join(output_dir, f"scene_{i:04d}.mp4")
        
        # Skip zero-duration segments
        if start_time >= end_time:
            print(f"Skipping zero-duration segment: start={start_time}, end={end_time}")
            continue
        
        # Re-encode the video to ensure proper playback
        cut_cmd = [
            "ffmpeg", "-y",
            "-i", video_path,
            "-ss", str(start_time),
            "-to", str(end_time),
            "-c:v", "libx264",  # Re-encode video using H.264 codec
            "-preset", "fast",  # Use a fast preset for encoding
            "-crf", "23",       # Set quality (lower is better, 23 is default)
            "-c:a", "aac",      # Re-encode audio using AAC codec
            "-b:a", "128k",     # Set audio bitrate
            out_path
        ]
        subprocess.run(cut_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
    
    return scenes

In [76]:
ffmpeg_scene_detection(video_path=video_path)

Detected scenes: [0.0, 0.500501, 2.502503, 3.503504, 3.545212, 5.505506, 10.01001, 10.510511, 13.513514, 17.017017, 23.940566]


[0.0,
 0.500501,
 2.502503,
 3.503504,
 3.545212,
 5.505506,
 10.01001,
 10.510511,
 13.513514,
 17.017017,
 23.940566]